In [0]:
dbutils.fs.rm('/user/hive/warehouse/tabelaclubes', recurse=True)

file_location = '/FileStore/tables/Tabela_Clubes.csv'
file_type = 'csv'
infer_schema = 'true'
first_row_is_header = 'true'
delimiter = ','

df = spark\
    .read\
    .format(file_type)\
    .option('inferSchema', infer_schema)\
    .option('header', first_row_is_header)\
    .option('sep', delimiter)\
    .load(file_location)

table_name = 'TabelaClubes'
    
df.write.format('parquet').saveAsTable(table_name)

Para esta atividade iremos responder algumas perguntas utilizando essas duas bases.

O Valor do elenco influencia nas primeiras/ultimas colocações

## 1 - O Valor do elenco influencia nas primeiras colocações?
##### Normalmente quem investe mais, tem um elenco mais qualificado para disputar o título.

In [0]:
%sql
with cte as (SELECT row_number()over(PARTITION BY ano order by Valor_total desc) as ID_Valor_elenco
,* 
FROM  default.tabelaclubes 
ORDER BY ANO DESC,Valor_total desc)

select `Pos.` as Posicao
,count(*) as `Qtd de Times `
from cte 
where ID_Valor_elenco = 1
group by `Pos.`
order by 2 desc

Posicao,Qtd de Times
1,3
4,2
13,1
3,1
10,1
9,1
2,1


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df = spark.table("default.tabelaclubes")

#definindo o valor do elenco 

df_vle = Window.partitionBy("ano").orderBy(F.desc("Valor_total"))

df_vle_id = df.withColumn("ID_Valor_elenco", F.row_number().over(df_vle))

result = df_vle_id.filter(df_vle_id["ID_Valor_elenco"] == 1)

result_grouped = result.groupBy(F.col("`Pos.`")).agg(F.count("*").alias("count")).orderBy(F.desc("count"))

# Exibindo o resultado
result_grouped.display()

Pos.,count
1,3
4,2
13,1
3,1
10,1
9,1
2,1


#### Podemos notar que nesses 10 anos o elenco mais caro foi campeão em apenas 3, representando apenas 30% do resultado.

## 2 - O Valor do elenco influencia nas últimas colocações?
##### Normalmente subtende-se que ter um elenco mais valioso muitas vezes são jogadores mais qualificados ou mais promissores.

In [0]:
%sql
SELECT row_number()over(PARTITION BY ano order by Valor_total asc) as ID_Valor_elenco
,* 
FROM  default.tabelaclubes 
ORDER BY ANO DESC,Valor_total asc

ID_Valor_elenco,Ano,Pos.,Clubes,Vitorias,Derrotas,Empates,GolsF/S,Saldo,Qtd_Jogadores,Idade_Media,Estrangeiros,Valor_total,Media_Valor,_c13,_c14,_c15,_c16
1,2017,20,Parana,4,11,23,18:57,-39,60,"22,9",3,18480000,308000,null,null,null,null
2,2017,9,Botafogo,13,12,13,38:46,-8,45,"23,1",4,25550000,568000,null,null,null,null
3,2017,15,Ceara,10,14,14,32:38,-6,54,26,2,26950000,499000,null,null,null,null
4,2017,18,America-MG,10,10,18,30:47,-17,51,"24,8",0,27350000,536000,null,null,null,null
5,2017,12,Fluminense,12,9,17,32:46,-14,58,"22,5",4,30800000,531000,null,null,null,null
6,2017,17,Sport,11,9,18,35:57,-22,47,"24,1",1,30950000,659000,null,null,null,null
7,2017,14,Chapecoense,11,11,16,34:50,-16,52,"24,8",5,32030000,616000,null,null,null,null
8,2017,11,Bahia,12,12,14,39:41,-2,48,"23,2",2,34900000,727000,null,null,null,null
9,2017,7,Athletico-PR,16,9,13,54:37,17,52,24,3,37650000,724000,null,null,null,null
10,2017,16,Vasco,10,13,15,41:48,-7,55,"23,9",7,41580000,756000,null,null,null,null


In [0]:
%sql
with cte as (SELECT row_number()over(PARTITION BY ano order by Valor_total asc) as ID_Valor_elenco
,* 
FROM  default.tabelaclubes 
ORDER BY ANO DESC,Valor_total asc)

select `Pos.` as Posicao
,count(*) as `Qtd de Times `
,Ano
,ID_Valor_elenco as `Ranking de time que menos Investiu no ano`
from cte 
where ID_Valor_elenco <5 and `Pos.` > 16
group by `Pos.`,ano,ID_Valor_elenco
order by 2 desc

Posicao,Qtd de Times,Ano,Ranking de time que menos Investiu no ano
18,1,2008,3
20,1,2009,2
18,1,2009,3
19,1,2010,1
19,1,2011,4
17,1,2012,2
19,1,2012,3
20,1,2014,1
19,1,2014,3
19,1,2015,1


In [0]:
%sql
SELECT row_number()over(PARTITION BY ano order by Valor_total asc) as ID_Valor_elenco
,* 
FROM  default.tabelaclubes 
ORDER BY ANO DESC,Valor_total asc


ID_Valor_elenco,Ano,Pos.,Clubes,Vitorias,Derrotas,Empates,GolsF/S,Saldo,Qtd_Jogadores,Idade_Media,Estrangeiros,Valor_total,Media_Valor,_c13,_c14,_c15,_c16
1,2017,20,Parana,4,11,23,18:57,-39,60,"22,9",3,18480000,308000,null,null,null,null
2,2017,9,Botafogo,13,12,13,38:46,-8,45,"23,1",4,25550000,568000,null,null,null,null
3,2017,15,Ceara,10,14,14,32:38,-6,54,26,2,26950000,499000,null,null,null,null
4,2017,18,America-MG,10,10,18,30:47,-17,51,"24,8",0,27350000,536000,null,null,null,null
5,2017,12,Fluminense,12,9,17,32:46,-14,58,"22,5",4,30800000,531000,null,null,null,null
6,2017,17,Sport,11,9,18,35:57,-22,47,"24,1",1,30950000,659000,null,null,null,null
7,2017,14,Chapecoense,11,11,16,34:50,-16,52,"24,8",5,32030000,616000,null,null,null,null
8,2017,11,Bahia,12,12,14,39:41,-2,48,"23,2",2,34900000,727000,null,null,null,null
9,2017,7,Athletico-PR,16,9,13,54:37,17,52,24,3,37650000,724000,null,null,null,null
10,2017,16,Vasco,10,13,15,41:48,-7,55,"23,9",7,41580000,756000,null,null,null,null


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df = spark.table("default.tabelaclubes")

# Definindo o valor do elenco
df_vle = Window.partitionBy("ano").orderBy(F.asc("Valor_total"))

df_vle_id = df.withColumn("ID_Valor_elenco", F.row_number().over(df_vle))

# Filtrando os dados onde ID_Valor_elenco é maior que 16
result = df_vle_id.filter(df_vle_id["ID_Valor_elenco"] < 5)

# Agrupando por `Pos.`, `ano` e `ID_Valor_elenco` e fazendo o COUNT
result_grouped = result.groupBy(F.col("`Pos.`"), F.col("ano"), F.col("ID_Valor_elenco")) \
    .agg(F.count("*").alias("count")) \
    .orderBy(F.desc("count"))


# Selecionando apenas as colunas que você deseja exibir
result_selected = result_grouped.select("`Pos.`", "ano", "count","ID_Valor_elenco")  # Selecionando as colunas

result_selected = result_selected \
    .withColumnRenamed("`Pos.`", "Posicao") \
    .withColumnRenamed("ano", "Ano") \
    .withColumnRenamed("count", "Quantidade de Times") \
    .withColumnRenamed("ID_Valor_elenco", "Ranking de Time que menos investiu")

# Exibindo o resultado com as colunas selecionadas
result_selected.filter(F.col("`Pos.`") > 16).display()

Pos.,Ano,Quantidade de Times,Ranking de Time que menos investiu
18,2008,1,3
20,2009,1,2
18,2009,1,3
19,2010,1,1
19,2011,1,4
17,2012,1,2
19,2012,1,3
20,2014,1,1
19,2014,1,3
19,2015,1,1



#### Podemos notar que nesses 10 anos os times que menos investiram, pelo menos 1 deles foi rebaixado, com excessão de 2013
